In [1]:
import os

import torch

import numpy as np
import matplotlib.pyplot as plt

from descwl_shear_sims.galaxies import FixedGalaxyCatalog
from descwl_shear_sims.galaxies import WLDeblendGalaxyCatalog
from descwl_shear_sims.stars import StarCatalog

from descwl_shear_sims.sim import make_sim

from descwl_shear_sims.psfs import make_fixed_psf
from descwl_shear_sims.psfs import make_ps_psf

from descwl_shear_sims.sim import get_se_dim

1. Generate images and catalogs

In [2]:
seed = 8312
rng = np.random.RandomState(seed)

ntrial = 2
coadd_dim = 351
buff = 50

for trial in range(ntrial):
    print('trial: %d/%d' % (trial+1, ntrial))

    # galaxy catalog; you can make your own
    galaxy_catalog = FixedGalaxyCatalog(
        rng=rng,
        coadd_dim=coadd_dim,
        buff=buff,
        layout='random',
        mag=25,
        hlr=1.0,
    )

    # make a constant gaussian psf
    psf = make_fixed_psf(psf_type='gauss')

    # generate some simulation data, with a particular shear

    sim_data = make_sim(
        rng=rng,
        galaxy_catalog=galaxy_catalog,
        coadd_dim=coadd_dim,
        g1=0.02,
        g2=0.00,
        bands = ['r', 'i', 'z'],
        psf=psf,
    )

trial: 1/2
trial: 2/2


2. Save the images and catalogs as tensors

In [3]:
images = torch.tensor(sim_data['band_data']['r'][0].image.array)

In [4]:
catalog = torch.zeros(1)

3. Save the tensors to `/data/scratch/weak_lensing/descwl`

In [5]:
torch.save(images, "/data/scratch/weak_lensing/descwl/images.pt")
torch.save(catalog, "/data/scratch/weak_lensing/descwl/catalog.pt")